# Pre screening

<b> Purpose:</b>

In [ ]:
from dotenv import load_dotenv

%pip install pandas
%pip install sqlalchemy>=2.0
%pip install oracledb
%pip install dotenv
%pip install psycopg2-binary
%pip install openpyxl

load_dotenv()


In [ ]:
from sqlalchemy import create_engine
import os
import pandas as pd

connect_to_db = 'postgresql://' + \
                os.getenv('DATABASE_USERNAME_COLIN_MIGR', '') + ":" + os.getenv('DATABASE_PASSWORD_COLIN_MIGR', '') +'@' + \
                os.getenv('DATABASE_HOST_COLIN_MIGR', '') + ':' + os.getenv('DATABASE_PORT_COLIN_MIGR', '5432') + '/' + \
                os.getenv('DATABASE_NAME_COLIN_MIGR', '');
                
engine = create_engine(connect_to_db)

# quick smoke test
with engine.connect() as conn:
    print(f"Connected to: {engine.url}")




In [ ]:
corps_pre_screening_query = f"""
select
    group_name,
    email_domain,
    admin_email,
    email_used_count,
    corp_num,
    corp_name,
    corp_type_cd,
    is_active,
    meets_main_criteria,
    recognition_dts,
    last_ar_filed_dt,
    director_count,
    filing_cnt,
    months_since_last_ar_filing,
    ar_unfiled_over_1yr,
    file_cnt_last_2yrs,
    last_file_event_ts,
    event_file_types,
    has_officers,
    has_3rd_party,
    vendor,
    migrated,
    has_password,
    send_ar_ind,
    address_all_any_bad_count,
    office_all_any_bad_count,
    party_all_any_bad_count,
    is_frozen
from mv_legacy_corps_data
where 1 = 1
    and corp_num in ('BC1000158','BC1113747','BC0968587','BC1279124','BC1101747','BC1101782','BC1101754','BC1457507',
    'BC1457472','BC1457468','BC1341761','BC1124231','BC0867774','BC0818117','BC0984688','BC0926240','BC1021743',
    'BC1023267','BC1101781','BC1063429','BC1101760','BC1101756','BC1101793','BC1101790','BC1101784','BC1101777',
    'BC1101774','BC1101771','BC1101779','BC1101746','BC1101788','BC1235739','BC1235742','BC1281425','BC1340203',
    'BC1524599','BC1543071','BC1487014','BC1509768','BC1000965','BC1145027','BC1023877','BC1155501','BC1036694',
    'BC1113308','BC1246338','BC1194740','BC1143575','BC1210015','BC1277546','BC1376919','BC1437993','BC1503663',
    'BC1437992','BC1504129','BC1531296','BC1528645','BC1098507','BC1191586','BC1234966','BC1191589','BC1138880')
;
"""

all_corps = pd.read_sql(corps_pre_screening_query, engine)
# filter rows where has_officers is False and meets_main_criteria is True
corps_has_officers = all_corps.query("has_officers == True")
corps_do_not_meets_criteria = all_corps.query("meets_main_criteria == False")
corps_null_group_name = all_corps.query("group_name.isnull()")
corps_migrated = all_corps.query("migrated == True")
corps_all_address_issues = all_corps.query("address_all_any_bad_count > 0")
corps_all_office_issues = all_corps.query("office_all_any_bad_count > 0")
corps_all_party_issues = all_corps.query("party_all_any_bad_count > 0")

print('All corps:', len(all_corps))
print('Has officers:', len(corps_has_officers))
print('Does not meet criteria:', len(corps_do_not_meets_criteria))
print('Null group name:', len(corps_null_group_name))
print('Migrated:', len(corps_migrated))
print('All address issues:', len(corps_all_address_issues))
print('All office issues:', len(corps_all_office_issues))
print('All party issues:', len(corps_all_party_issues))